In [274]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from asset_data_module import read_close_prices_all_merged
from features import make_feature_windows
tf.keras.utils.set_random_seed(42)


In [275]:
markets = ['dow30']
# markets = ['commodities']
# markets = ['dow30', 'commodities', 'bonds', 'funds_mini']
# start_date, end_date = "2022-01-01", "2025-11-28"
start_date, end_date = "2024-09-01", "2025-11-28"

_, close_df = read_close_prices_all_merged(markets, after_date=start_date)
close_df = close_df.loc[:end_date]

rolling = make_feature_windows(
    close_prices=close_df,
    lookback=30,
    horizon=1,
    days_per_week=2
)
close_df.shape, len(rolling)

((312, 30), 126)

In [276]:
def panel_from_windows(windows, x_key="past_weekly_returns", y_key="y_ret", residual_over_sample_mean=True, baseline_key="past_weekly_returns"):
    X_list, y_list, mu_list = [], [], []
    meta_rows = []

    for w_idx, w in enumerate(windows):

        # ---- X ----
        if x_key == "past_weekly_returns":
            X_df = w[x_key].T                 # assets x lookback
        elif x_key == "X_feat":
            X_df = w[x_key]                   # assets x features
        else:
            raise ValueError(f"Unknown x_key: {x_key}")

        # ---- y ----
        y_ser = w[y_key]
        if not isinstance(y_ser, pd.Series):
            y_ser = pd.Series(y_ser, index=X_df.index)

        # ---- baseline mu (always computed from past period returns) ----
        # past_weekly_returns: (lookback periods) x assets  -> mean over rows gives Series(index=assets)
        mu_ser = w[baseline_key].mean(axis=0)
        if not isinstance(mu_ser, pd.Series):
            mu_ser = pd.Series(mu_ser, index=w[baseline_key].columns)

        # ---- align ----
        assets = X_df.index.intersection(y_ser.index).intersection(mu_ser.index)
        Xw = X_df.loc[assets].to_numpy(np.float32)
        yw = y_ser.loc[assets].to_numpy(np.float32)
        muw = mu_ser.loc[assets].to_numpy(np.float32)

        # ---- make residual target if requested ----
        if residual_over_sample_mean:
            yw = yw - muw   # residual = actual - baseline

        mask = np.isfinite(Xw).all(axis=1) & np.isfinite(yw) & np.isfinite(muw)
        Xw, yw, muw = Xw[mask], yw[mask], muw[mask]
        assets_kept = assets.to_numpy()[mask]

        X_list.append(Xw)
        y_list.append(yw)
        mu_list.append(muw)

        t0, t1 = w.get("t0", None), w.get("t1", None)
        for a in assets_kept:
            meta_rows.append((w_idx, a, t0, t1))

    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    mu = np.concatenate(mu_list)
    meta = pd.DataFrame(meta_rows, columns=["window_idx", "asset", "t0", "t1"])
    return X, y, mu, meta

In [277]:
X_past_returns, _, _, _ = panel_from_windows(rolling, x_key='past_weekly_returns')
# X, y, mu, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
X, y, mu, meta = panel_from_windows(rolling, x_key='X_feat')
X.shape, y.shape, mu.shape, meta.head() ## len(rolling)*n_asset -- each row is a feature set -- to predict y

((3780, 9),
 (3780,),
 (3780,),
    window_idx       asset         t0         t1
 0           0  dow30:AAPL 2024-11-25 2024-11-27
 1           0  dow30:AMGN 2024-11-25 2024-11-27
 2           0   dow30:AXP 2024-11-25 2024-11-27
 3           0    dow30:BA 2024-11-25 2024-11-27
 4           0   dow30:CAT 2024-11-25 2024-11-27)

In [278]:
windows = rolling  # your rolling list

W = meta["window_idx"].nunique()
split_w = int(0.8 * W)

train_mask = (meta["window_idx"] < split_w).values
test_mask  = (meta["window_idx"] >= split_w).values

X_train_raw, y_train = X[train_mask], y[train_mask]
X_test_raw,  y_test  = X[test_mask],  y[test_mask]
X_past_returns_test_raw = X_past_returns[test_mask]
mu_train = mu[train_mask]
mu_test  = mu[test_mask]


# small validation from the tail of the training windows
val_w = max(int(0.1 * split_w), 1)
val_start = split_w - val_w
val_mask = ((meta["window_idx"] >= val_start) & (meta["window_idx"] < split_w)).values
tr2_mask = (meta["window_idx"] < val_start).values

X_tr_raw, y_tr = X[tr2_mask], y[tr2_mask]
X_va_raw, y_va = X[val_mask], y[val_mask]

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr_raw).astype(np.float32)
X_va = scaler.transform(X_va_raw).astype(np.float32)
X_te = scaler.transform(X_test_raw).astype(np.float32)

y_tr = y_tr.astype(np.float32)
y_va = y_va.astype(np.float32)
y_te = y_test.astype(np.float32)

mu_tr = mu[tr2_mask]
mu_va = mu[val_mask]

X_tr.shape, X_va.shape, X_te.shape, y_tr.shape, y_va.shape, y_te.shape, mu_tr.shape, mu_va.shape, mu_test.shape, f"{y_te.shape[0]/close_df.shape[1]} test periods"

((2700, 9),
 (300, 9),
 (780, 9),
 (2700,),
 (300,),
 (780,),
 (2700,),
 (300,),
 (780,),
 '26.0 test periods')

In [279]:
def build_mlp(in_dim, hidden=(64, 32), dropout=0.1, lr=1e-3):
    inputs = keras.Input(shape=(in_dim,))
    x = inputs
    for h in hidden:
        x = layers.Dense(h, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_mlp(in_dim=X_tr.shape[1], hidden=(16, 8), dropout=0.1, lr=3e-4)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )
]

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=150,
    batch_size=256,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.2890 - mae: 0.3948 - val_loss: 0.1817 - val_mae: 0.3174
Epoch 2/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2531 - mae: 0.3641 - val_loss: 0.1513 - val_mae: 0.2862
Epoch 3/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2216 - mae: 0.3383 - val_loss: 0.1275 - val_mae: 0.2606
Epoch 4/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2008 - mae: 0.3193 - val_loss: 0.1090 - val_mae: 0.2391
Epoch 5/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1689 - mae: 0.2954 - val_loss: 0.0950 - val_mae: 0.2218
Epoch 6/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1522 - mae: 0.2783 - val_loss: 0.0841 - val_mae: 0.2076
Epoch 7/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1507 - mae: 0.2708 - val_loss: 0.0750 - val_mae: 0.1954
Epoch 8/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1307 - mae: 0.2543 - val_loss: 0.0676 - val_mae: 0.1852
Epoch 9/150
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.11

In [280]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

def prediction_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    sign_acc = (np.sign(y_true) == np.sign(y_pred)).mean()
    corr = np.corrcoef(y_true, y_pred)[0, 1] if len(y_true) > 1 else np.nan
    
    return {"MSE": round(mse, 5), "MAE": round(mae, 5), "R2": round(r2, 5), "SignAcc": round(float(sign_acc), 5), "Corr": round(float(corr), 5)}


In [281]:
return_true_test = y_te + mu_test

y_pred_sm = mu_test
print("Sample Mean")
print(prediction_metrics(return_true_test, y_pred_sm))

y_pred_residual = model.predict(X_te, batch_size=1024).squeeze()
return_pred = mu_test + y_pred_residual
print("MLP")
print(prediction_metrics(return_true_test, return_pred))

print(y_te.mean())
print(return_true_test.mean())
print(y_pred_residual.mean())

Sample Mean
{'MSE': 0.00039, 'MAE': 0.01341, 'R2': -0.06373, 'SignAcc': 0.49359, 'Corr': -0.025}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
MLP
{'MSE': 0.00078, 'MAE': 0.01902, 'R2': -1.15339, 'SignAcc': 0.47564, 'Corr': 0.00916}
-0.00031272165
0.0011933894
0.004373363


In [282]:
y_true_test = y_te + mu_test
print("y_true_test quantiles:", np.quantile(y_true_test, [0, .01, .5, .99, 1]))
print("mu_test quantiles    :", np.quantile(mu_test, [0, .01, .5, .99, 1]))

y_true_test quantiles: [-0.08066442 -0.04359215  0.000701    0.05251097  0.20515133]
mu_test quantiles    : [-0.01007372 -0.00743506  0.00090595  0.01633628  0.02404095]


Walk-Forward For Residual Prediction

In [283]:
def baseline_sample_mean(test_window, baseline_key="past_weekly_returns"):
    # returns Series indexed by asset
    return test_window[baseline_key].mean(axis=0)

def walk_forward_eval_mlp_residual(
    rolling,
    train_len=150,
    x_key="X_feat",
    y_key="y_ret",
    baseline_key="past_weekly_returns",   # where to compute mu from
    hidden=(64, 32),
    dropout=0.1,
    lr=3e-4,
    epochs=150,
    batch_size=256,
    seed=42
):
    tf.keras.utils.set_random_seed(seed)

    all_rows = []
    period_metrics = []

    for i in range(train_len, len(rolling)):
        train_windows = rolling[i-train_len:i]
        test_window   = rolling[i]

        # --- TRAIN panel: residual target ---
        X_train, y_res_train, mu_train, meta_train = panel_from_windows(
            train_windows,
            x_key=x_key,
            y_key=y_key,
            residual_over_sample_mean=True,
            baseline_key=baseline_key,
        )
        if X_train.shape[0] == 0:
            continue

        # --- scale X on TRAIN only ---
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train).astype(np.float32)

        # --- build TEST cross-section ---
        X_test_df = test_window[x_key]  # assets x features
        if x_key == 'past_weekly_returns':
            X_test_df = X_test_df.T
        y_test = test_window[y_key]
        if not isinstance(y_test, pd.Series):
            y_test = pd.Series(y_test, index=X_test_df.index)

        mu_ser = baseline_sample_mean(test_window, baseline_key=baseline_key)

        # align assets for X, y_true, mu
        assets = X_test_df.index.intersection(y_test.index).intersection(mu_ser.index).sort_values()

        X_test = X_test_df.loc[assets].to_numpy(np.float32)
        y_true = y_test.loc[assets].to_numpy(np.float32)      # TRUE return
        mu_test = mu_ser.loc[assets].to_numpy(np.float32)     # baseline return forecast

        mask = np.isfinite(X_test).all(axis=1) & np.isfinite(y_true) & np.isfinite(mu_test)
        assets = assets[mask]
        X_test = X_test[mask]
        y_true = y_true[mask]
        mu_test = mu_test[mask]

        X_test_sc = scaler.transform(X_test).astype(np.float32)

        # --- train fresh model each step ---
        model = build_mlp(in_dim=X_train_sc.shape[1], hidden=hidden, dropout=dropout, lr=lr)
        es = keras.callbacks.EarlyStopping(monitor="loss", patience=15, restore_best_weights=True)

        model.fit(
            X_train_sc, y_res_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[es]
        )

        # --- predict residual, reconstruct return ---
        pred_res = model.predict(X_test_sc, batch_size=1024, verbose=0).squeeze()
        y_pred_return = mu_test + pred_res

        # --- baseline prediction (sample mean) ---
        y_pred_sm = mu_test

        # store per-asset predictions
        for a, yt, yhat, yhat_sm, muhat, rhat in zip(assets, y_true, y_pred_return, y_pred_sm, mu_test, pred_res):
            all_rows.append({
                "window_idx": i,
                "t0": test_window.get("t0", None),
                "t1": test_window.get("t1", None),
                "asset": a,
                "y_true": float(yt),
                "mu": float(muhat),
                "pred_res": float(rhat),
                "pred_mlp": float(yhat),
                "pred_sm": float(yhat_sm),
            })

        # per-window metrics (cross-section) on TRUE returns
        m_mlp = prediction_metrics(y_true, y_pred_return)
        m_sm  = prediction_metrics(y_true, y_pred_sm)

        period_metrics.append({
            "window_idx": i,
            "t0": test_window.get("t0", None),
            "t1": test_window.get("t1", None),
            **{f"mlp_{k}": v for k, v in m_mlp.items()},
            **{f"sm_{k}": v for k, v in m_sm.items()},
        })
        
        print("X_train raw min/max:", X_train.min(), X_train.max())
        print("y_res_train min/max:", y_res_train.min(), y_res_train.max())
        print("mu_train min/max:", mu_train.min(), mu_train.max())
    
    preds_df = pd.DataFrame(all_rows)
    period_df = pd.DataFrame(period_metrics)

    pooled_mlp = prediction_metrics(preds_df["y_true"], preds_df["pred_mlp"]) if len(preds_df) else {}
    pooled_sm  = prediction_metrics(preds_df["y_true"], preds_df["pred_sm"])  if len(preds_df) else {}
    

    return preds_df, period_df, pooled_mlp, pooled_sm

In [284]:
preds_df, period_df, pooled_mlp, pooled_sm = walk_forward_eval_mlp_residual(
    rolling=rolling,
    train_len=100,
    x_key="X_feat",
    # x_key="past_weekly_returns",
    y_key="y_ret",
    baseline_key="past_weekly_returns",
    hidden=(16,8)
)

print("Pooled MLP:", pooled_mlp)
print("Pooled  SM:", pooled_sm)
print("Win-rate (MAE):", (period_df["mlp_MAE"] < period_df["sm_MAE"]).mean())

X_train raw min/max: -5.2934895 5.294527
y_res_train min/max: -0.18489625 0.13687612
mu_train min/max: -0.02251788 0.014512167
X_train raw min/max: -5.2934895 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.014512167
X_train raw min/max: -5.2934895 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.014512167
X_train raw min/max: -5.2934895 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.014512167
X_train raw min/max: -5.292356 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.014512167
X_train raw min/max: -5.292356 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.015215398
X_train raw min/max: -5.292356 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 0.015215398
X_train raw min/max: -5.292356 5.294527
y_res_train min/max: -0.18489625 0.19928172
mu_train min/max: -0.02251788 

In [285]:
print("X_train raw min/max:", X_tr.min(), X_tr.max())
print("y_res_train min/max:", y_tr.min(), y_tr.max())
print("mu_train min/max:", mu_tr.min(), mu_tr.max())

X_train raw min/max: -5.383983 5.38163
y_res_train min/max: -0.18489625 0.13687612
mu_train min/max: -0.02251788 0.014512167
